# Kite Packaging

## Initial Exploratory Data Analysis

In [1]:
# Import the necessary libraries and packages.
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
# Load the CSV files as dataframes (customers, orders and order_details).
customers = pd.read_csv('kite_customers.csv')
orders = pd.read_csv('orders.csv')
order_details = pd.read_csv('order_details.csv')
vehicles = pd.read_csv('vehicles.csv')

### Explore the customers data frame

In [3]:
# View the customers dataframe.
customers.head()

,WebsiteID,DeliveryPostCode,DeliveryCountry
0,245,L39 2,United Kingdom
1,597,PR3 1,United Kingdom
2,863,PE2 9,United Kingdom
3,2180,LN5 8,United Kingdom
4,2644,BD10 9,United Kingdom


In [4]:
# View data types, nulls and columns for the customers dataframe.
customers.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 18727 entries, 0 to 18726
Data columns (total 3 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   WebsiteID         18727 non-null  int64 
 1   DeliveryPostCode  18662 non-null  object
 2   DeliveryCountry   18662 non-null  object
dtypes: int64(1), object(2)
memory usage: 439.0+ KB


In [7]:
# Count the number of unique customerID values.
customers['WebsiteID'].nunique()

18727

- There are 18,727 unique websiteID values, indicating 18,727 different customers.

In [8]:
# Count the number of unique customerID values.
customers['DeliveryPostCode'].nunique()

7040

- There are 7040 unique post codes in the database. Some postcodes will have more than one customer.

In [10]:
# Count the number of unique customerID values.
customers['DeliveryCountry'].value_counts()

United Kingdom     18654
Ireland                7
United Kingdom         1
Name: DeliveryCountry, dtype: int64

- All but seven deliveries are to the UK. There is one record where UK may have an extra space - needs to be cleaned.

### Explore the orders data frame

In [11]:
# View the orders dataframe.
orders.head()

,OrderID,WebsiteID,OrderDate,StatusName,StatusId,PaymentOptionName,PaymentType,DespatchedCourier,ProductsExVAT,DeliveryExVAT,VAT,SubTotal,GrandTotal,DateDespatched,DeliveryCost
0,2275137,852084,27/03/2023 00:04,Dispatched,6,Paypal,1,DX Freight,103.05,0.00,20.62,103.05,123.67,27/03/2023 00:00,19.17
1,2275138,673961,27/03/2023 00:21,Dispatched,6,On Account,3,MFS,480.86,0.00,96.18,480.86,577.04,27/03/2023 00:00,76.73
2,2275139,679319,27/03/2023 00:22,Dispatched,6,Payment Gateway,2,DX Freight,125.28,0.00,25.05,125.28,150.33,27/03/2023 00:00,22.67
3,2275140,819078,27/03/2023 00:54,Dispatched,6,Paypal,1,DX Express,21.26,7.22,5.69,21.26,34.17,27/03/2023 00:00,6.81
4,2275141,692729,27/03/2023 00:56,Dispatched,6,Paypal,1,DX Express,127.88,0.00,25.57,127.88,153.45,27/03/2023 00:00,11.19


In [12]:
# View data types, nulls and columns for the customers dataframe.
orders.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 24385 entries, 0 to 24384
Data columns (total 15 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   OrderID            24385 non-null  int64  
 1   WebsiteID          24385 non-null  int64  
 2   OrderDate          24385 non-null  object 
 3   StatusName         24385 non-null  object 
 4   StatusId           24385 non-null  int64  
 5   PaymentOptionName  24385 non-null  object 
 6   PaymentType        24385 non-null  int64  
 7   DespatchedCourier  24385 non-null  object 
 8   ProductsExVAT      24385 non-null  float64
 9   DeliveryExVAT      24385 non-null  float64
 10  VAT                24385 non-null  float64
 11  SubTotal           24385 non-null  float64
 12  GrandTotal         24385 non-null  float64
 13  DateDespatched     23410 non-null  object 
 14  DeliveryCost       23676 non-null  float64
dtypes: float64(6), int64(4), object(5)
memory usage: 2.8+ MB


- The OrderDate and DateDespatched columns may need to be converted to a date format.
- There are some null values in DeliveryCost and DateDespatched, which may need to be removed.

In [13]:
# Count the number of unique customerID values.
orders['DespatchedCourier'].value_counts()

DX Express    9935
DX Freight    9890
MFS           4073
Split          324
Direct         163
Name: DespatchedCourier, dtype: int64

- Most deliveries are sent via DX Express or Freight. A small number are Direct.
- Simon mentioned 3 third parties - I assume they are DX Express, DX Freight and MFS.
- He also mentioned they were for pallets, parcels and ugly freight.
- We should confirm that each courier deals with one type of freight.

In [16]:
# Decribe the variables in orders.
orders.describe().round()

,OrderID,WebsiteID,StatusId,PaymentType,ProductsExVAT,DeliveryExVAT,VAT,SubTotal,GrandTotal,DeliveryCost
count,24385.0,24385.0,24385.0,24385.0,24385.0,24385.0,24385.0,24385.0,24385.0,23676.0
mean,2287329.0,632149.0,6.0,2.0,238.0,5.0,48.0,238.0,290.0,25.0
std,7039.0,238482.0,1.0,1.0,416.0,9.0,83.0,416.0,495.0,39.0
min,2275137.0,245.0,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
25%,2281233.0,547099.0,6.0,2.0,59.0,0.0,14.0,59.0,82.0,7.0
50%,2287329.0,678347.0,6.0,2.0,127.0,0.0,26.0,127.0,154.0,13.0
75%,2293425.0,830781.0,6.0,3.0,259.0,7.0,52.0,259.0,314.0,31.0
max,2299521.0,901736.0,100.0,4.0,12987.0,662.0,2598.0,12987.0,15585.0,1297.0


### Explore the order_details data frame

In [17]:
# View the orders dataframe.
order_details.head()

,OrderID,PartNumber,Description,Packs,PackPrice,UnitsPerPack,PackWeight,LineWeight,PalletQuantity,CubicMPerUnit
0,2275137,CDW241818M,Double wall boxes - 24x18x18 inches (610x457x4...,1,52.44,20,20.98,20.98,140,0.009683
1,2275137,CSW090606M-P30,Single wall cardboard boxes - 9x6x6 inches (22...,1,7.20,30,2.11,2.11,3600,0.000435
2,2275137,CSW120909M,Single wall cardboard boxes - 12x9x9 inches (3...,1,10.38,30,5.04,5.04,1800,0.000910
3,2275137,CSW120906M-P30,Single wall cardboard boxes - 12x9x6 inches (3...,1,8.08,25,3.65,3.65,2100,0.000911
4,2275137,CDW181212M,Double wall boxes - 18x12x12 inches (457x305x3...,1,24.95,20,9.02,9.02,560,0.004687


In [18]:
# View data types, nulls and columns for the customers dataframe.
order_details.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 60818 entries, 0 to 60817
Data columns (total 10 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   OrderID         60818 non-null  int64  
 1   PartNumber      60818 non-null  object 
 2   Description     60818 non-null  object 
 3   Packs           60818 non-null  int64  
 4   PackPrice       60818 non-null  float64
 5   UnitsPerPack    60818 non-null  int64  
 6   PackWeight      60818 non-null  float64
 7   LineWeight      60684 non-null  float64
 8   PalletQuantity  60818 non-null  int64  
 9   CubicMPerUnit   57636 non-null  float64
dtypes: float64(4), int64(4), object(2)
memory usage: 4.6+ MB


## Initial thoughts and questions
- Are all the orders in these dataframes dispatched from Coventry?
- Create a subset dataframe which includes calculated values of total lineweight and total volume for each order (using a groupby function from the order details table.
- Looking at the postcodes using Tableau, it looks like these are all orders in a three week period, so we should be looking at which distribution centre should be dispatching which orders.